In [4]:
import geopandas as gpd
from shapely.geometry import Point
from geopy.distance import geodesic
from datetime import datetime, timedelta
import pandas as pd

In [5]:
# Create a GeoPandas DataFrame with sample GPS data including 'timestamp' and 'geometry' columns
data = pd.read_csv('D:/Extra-Projects/MGL-India/Moving Dots/trip_data_10Units_1Year_V2.csv',usecols=['UnityLicensePlate', 'RowReferenceTime', 'lat', 'lon','Speed','EngineStatus'],parse_dates=['RowReferenceTime'])
gdf = gpd.GeoDataFrame(data,geometry= gpd.points_from_xy(data['lat'],data['lon']), crs='EPSG:4326')

In [12]:
# Convert 'timestamp' column to datetime format
gdf['timestamp'] = pd.to_datetime(gdf['RowReferenceTime'],format='ISO8601')

# Function to calculate distance between two points using geopy
def calculate_distance(row,prev_row):
    point_a = (row['geometry'].x, row['geometry'].y)
    point_b = (prev_row['geometry'].x, prev_row['geometry'].y)  # Example using the first point as reference
    return geodesic(point_a, point_b).kilometers

# Function to filter rows based on time and distance criteria
def filter_rows(row):
    time_threshold = timedelta(minutes=5)

    if row.name > 0:
        time_difference = row['timestamp'] - gdf.loc[row.name - 1, 'timestamp']
        if time_difference < time_threshold:
            return False

        distance = calculate_distance(row, gdf.loc[row.name - 1])
        return distance <= 2.0

    # For the first row, always include it in the result
    return True

# Apply the filtering function to each row in the GeoDataFrame
filtered_gdf = gdf[gdf.apply(filter_rows, axis=1)]

In [11]:
# Print the resulting GeoDataFrame
print(filtered_gdf.reset_index(drop=True))

       UnityLicensePlate               RowReferenceTime       lat       lon  \
0               VIG26878  2023-11-01 07:27:33.697000000  19.36178 -98.95371   
1               VIG26878  2023-11-01 01:48:52.690000000  19.36144 -98.95451   
2               VIG26878  2023-11-01 07:27:16.697000000  19.36146 -98.95401   
3               VIG26878  2023-11-01 07:25:14.843000000  19.36136 -98.95452   
4               VIG26878  2023-11-01 07:29:30.717000000  19.36096 -98.95286   
...                  ...                            ...       ...       ...   
141315          CEV30321  2023-01-26 12:15:29.087000000  19.44023 -99.20637   
141316          CEV30321            2023-01-23 07:52:42  19.43237 -99.17939   
141317          VIG26878  2023-01-31 17:29:24.247000000  19.33344 -98.95144   
141318          VIG26878  2023-01-31 18:44:39.850000000  19.36135 -98.95444   
141319          VIG23896  2023-01-27 10:51:55.307000000  19.46822 -99.12085   

        Speed  EngineStatus                    geom